## Read file

In [ ]:
#!fsharp
open System.IO
let imageBuffer = File.ReadAllBytes "./images/passport-2.jpg"

## Convert image to base64 string 

In [ ]:
#!fsharp
open System
let image64 = Convert.ToBase64String imageBuffer

## Use yandex API to extract text from image

### Get yandex token first

Your current folder has been set to 'default' (id = b1g3aavp9tndrdr9q148).


[ya-ocr docs](https://cloud.yandex.com/en-ru/docs/vision/operations/ocr/text-detection)

!IMPORTNAT! 

Get the ID of any folder that your account is granted the editor role or higher for.

Add `editor` role on folder when request OCR API



In [ ]:
#!powershell
yc iam create-token

t1.9euelZqKz5mVyY2Oj5WSns_OjseSlO3rnpWal8ebi5aJmJ2Ky4yVkc2dmJrl8_dFH2t3-e9RFVx0_t3z9wVOaHf571EVXHT-.nz2n3v6l4oiKXYrkNc1De-7h0Exd7XdY-jrXxDZ-biJWHB0zVu4s955UcEE1C-6EHp6u16g06glnT-U-JXcWBw


### Extract data from the image using yandex API

In [ ]:
// Result type def
// created with https://app.quicktype.io/ convert to typescript and then modify manually
type Vertex = {
    x: string;
    y: string;
}

type Language = {
    languageCode: string;
    confidence:   float;
}

type BoundingBox = {
    vertices: Vertex[];
}

type Word = {
    boundingBox: BoundingBox;
    languages:   Language[];
    text:        string;
    confidence:  float;
}

type Line = {
    boundingBox: BoundingBox;
    words:       Word[];
    confidence:  float;
}

type Block = {
    boundingBox: BoundingBox;
    lines:       Line[];
}

type Page = {
    blocks: Block[];
    width:  string;
    height: string;
}

type TextDetection = {
    pages: Page[];
}

type ResultResult = {
    textDetection: TextDetection;
}

type YaOcrResult = {
    results: ResultResult[];
}

type YaOcr = {
    results: YaOcrResult[];
    code: int;
    message: string;
}



In [ ]:
#!fsharp

#r "nuget:Oryx"
#r "nuget:Oryx.SystemTextJson"

// create you yandex account first
let YA_FOLDER = "b1g3aavp9tndrdr9q148"
// copy value from output of pervious cell
let YA_TOKEN="t1.9euelZqKy5WRi4qSyZGMl8aMl8aPlu3rnpWazIuVipiey5mXiYuMz8qdi4nl9Pc8Fmt3-e8GGF2w3fT3fERod_nvBhhdsM3n9euelZqYlpfMm5qWm5vIjo6TnpjGjO_9.jB4x9evv-gX0xALDIwdpnSbZS9F8uYPCQfwgB1NvSAaZjOWMd8Y-nEN9fwtea-NtPb8V2am1LZP2NqjwdQhuAA"
let YA_URL="https://vision.api.cloud.yandex.net/vision/v1/batchAnalyze"

open Oryx
open Oryx.SystemTextJson.ResponseReader
open System.Net.Http
open System.Threading.Tasks
open System.Text.Json

let options = JsonSerializerOptions()

let bodyFeatures = 
    {|
        Type = "TEXT_DETECTION"
        text_detection_config = {| language_codes = [|"*"|] |}
    |}
    
let body = {|
    folderId = YA_FOLDER
    analyze_specs = [|
      {|
        content = image64
        features = bodyFeatures
      |}
    |]
|}


let bodyContent = Json.JsonContent.Create body :> HttpContent

let request = 
    POST 
    >=> withUrl YA_URL 
    >=> withContent (fun _ -> bodyContent)
    >=> fetch
    >=> json<YaOcr> options

let client = new HttpClient ()
let ctx = 
    HttpContext.defaultContext     
    |> HttpContext.withHeader ("Authorization", $"Bearer {YA_TOKEN}")
    |> HttpContext.withHttpClient client 

let yaOcrJsonResult = request |> runAsync ctx |> Async.AwaitTask |> Async.RunSynchronously

yaOcrJsonResult


ResultValue ErrorValue { { results =
 [|{ results =
 [|{ textDetection =
 { pages =
 [|{ blocks =
 [|{ boundingBox =
 { vertices =
 [|{ x = "122"
 y = "57" }; { x = "122"
 y = "73" }; { x = "368"
 y = "73" };
 { x = "368"
 y = "57" }|] }
 lines =
 [|{ boundingBox =
 { vertices =
 [|{ x = "122"
 y = "57" }; { x = "122"
 y = "73" }; { x = "368"
 y = "73" };
 { x = "368"
 y = "57" }|] }
 words =
 [|{ boundingBox =
 { vertices =
 [|{ x = "122"
 y = "55" }; { x = "122"
 y = "74" };
 { x = "368"
 y = "74" }; { x = "368"
 y = "55" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.9276171327 }|]
 text = "российская"
 confidence = 0.9276171327 }|]
 confidence = 0.9276171327 }|] };
 { boundingBox =
 { vertices =
 [|{ x = "404"
 y = "59" }; { x = "404"
 y = "78" }; { x = "617"
 y = "78" };
 { x = "617"
 y = "59" }|] }
 lines =
 [|{ boundingBox =
 { vertices =
 [|{ x = "404"
 y = "59" }; { x = "404"
 y = "78" }; { x = "617"
 y = "78" };
 { x = "617"
 y = "59" }|] }
 words =
 [|{ boundingBox =
 { vertices =
 [|{ x = "404"
 y = "57" }; { x = "404"
 y = "78" };
 { x = "557"
 y = "78" }; { x = "557"
 y = "57" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.8988859653 }|]
 text = "федерац"
 confidence = 0.8988859653 };
 { boundingBox =
 { vertices =
 [|{ x = "572"
 y = "61" }; { x = "572"
 y = "78" };
 { x = "589"
 y = "78" }; { x = "589"
 y = "61" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.8988859653 }|]
 text = "и"
 confidence = 0.8988859653 };
 { boundingBox =
 { vertices =
 [|{ x = "604"
 y = "61" }; { x = "604"
 y = "79" };
 { x = "617"
 y = "79" }; { x = "617"
 y = "61" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.8988859653 }|]
 text = "я"
 confidence = 0.8988859653 }|]
 confidence = 0.8988859653 }|] };
 { boundingBox =
 { vertices =
 [|{ x = "241"
 y = "98" }; { x = "241"
 y = "118" }; { x = "541"
 y = "118" };
 { x = "541"
 y = "98" }|] }
 lines =
 [|{ boundingBox =
 { vertices =
 [|{ x = "241"
 y = "98" }; { x = "241"
 y = "118" }; { x = "541"
 y = "118" };
 { x = "541"
 y = "98" }|] }
 words =
 [|{ boundingBox =
 { vertices =
 [|{ x = "241"
 y = "97" }; { x = "241"
 y = "117" };
 { x = "379"
 y = "117" }; { x = "379"
 y = "97" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.8886364102 }|]
 text = "ОТДЕЛЕНИЕМ"
 confidence = 0.8886364102 };
 { boundingBox =
 { vertices =
 [|{ x = "392"
 y = "99" }; { x = "392"
 y = "118" };
 { x = "447"
 y = "118" }; { x = "447"
 y = "99" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.8886364102 }|]
 text = "УФМС"
 confidence = 0.8886364102 };
 { boundingBox =
 { vertices =
 [|{ x = "460"
 y = "99" }; { x = "460"
 y = "119" };
 { x = "541"
 y = "119" }; { x = "541"
 y = "99" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.8886364102 }|]
 text = "РОССИИ"
 confidence = 0.8886364102 }|]
 confidence = 0.8886364102 }|] };
 { boundingBox =
 { vertices =
 [|{ x = "241"
 y = "138" }; { x = "241"
 y = "158" }; { x = "543"
 y = "158" };
 { x = "543"
 y = "138" }|] }
 lines =
 [|{ boundingBox =
 { vertices =
 [|{ x = "241"
 y = "138" }; { x = "241"
 y = "158" }; { x = "543"
 y = "158" };
 { x = "543"
 y = "138" }|] }
 words =
 [|{ boundingBox =
 { vertices =
 [|{ x = "241"
 y = "136" }; { x = "241"
 y = "156" };
 { x = "275"
 y = "156" }; { x = "275"
 y = "136" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.9176917672 }|]
 text = "по"
 confidence = 0.9176917672 };
 { boundingBox =
 { vertices =
 [|{ x = "288"
 y = "137" }; { x = "288"
 y = "158" };
 { x = "435"
 y = "158" }; { x = "435"
 y = "137" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.9176917672 }|]
 text = "челябинской"
 confidence = 0.9176917672 };
 { boundingBox =
 { vertices =
 [|{ x = "448"
 y = "139" }; { x = "448"
 y = "160" };
 { x = "543"
 y = "160" }; { x = "543"
 y = "139" }|] }
 languages = [|{ languageCode = "ru"
 confidence = 0.9176917672 }|]
 text = "области"
 confidence = 0.9176917672 }|]
 confidence = 0.9176917672 }|] };
 { boundi

In [ ]:
// extract text from result

let yaOcrJson = 
    match yaOcrJsonResult with 
    | Ok yaOcrJson -> yaOcrJson
    | Error err ->
        raise err

let blocks = yaOcrJson.results.[0].results.[0].textDetection.pages.[0].blocks

let words =
    blocks|> Array.collect(fun block -> block.lines |> Array.collect(fun line -> line.words |> Array.map(fun word -> word.text)))

words

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at <StartupCode$FSI_0012>.$FSI_0012.main@()

### Preapre data and utilities to cleanup words

In [ ]:
open System.IO

File.WriteAllLines("./out/passport-2-words.txt", words)